# Driver Drowsiness Detection System

Studies indicate that fatigue-related crashes account for about 20% of road accidents and even more on roads with no driving hour regulations. Driver detection systems, particularly those focusing on drowsiness detection, aim to address that alarming rate by monitoring drivers for signs of drowsiness and issuing timely alerts to prevent potential crashes.

For our final project, we chose to develop a DDS by using the UTA Real-Life Drowsiness Dataset, which features diverse participants and comprehensive data. We will train a convolutional neural network (CNN) to analyze facial, eye, and mouth movements at different stages of drowsiness. The model will provide warnings and alerts based on the detected level of fatigue, with accuracy tests ensuring its reliability.

### Requirements 
● TensorFlow: Developed by the Google Brain team for machine learning and artificial intelligence, Tensorflow has a allows for training and inference of deep neural networks.

● Keras: Provides a Python interface for artificial neural networks (inbuilt python library).

● Numpy: Used for scientific computing in Python. Provides support for arrays, matrices, and various mathematical functions to operate on them. 

● OpenCV: Machine learning and compiter vision library; contains >2500 algorhitms optimized for various CV tasks 

● Scikit-learn: Data mining, data analysis. In this project, used for splitting datasets. 

● Pandas: Data manipulation and analysis library. Used to create dataframes associating frames with their labels.

In [ ]:
# Importing required libraries
import numpy as np 
import pandas as pd 
import tensorflow as tg
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os

## Kaggle Adjustments 
Deletion of all previous files and checks for appropriate directories is performed. 

In [3]:
!rm -rf /kaggle/working/*
folder1 = "folder1"
folder2 = "folder2"

if not os.path.exists(folder1):
    os.mkdir(folder1)
if not os.path.exists(folder2):
    os.mkdir(folder2)

# Change directory
os.chdir('/kaggle/working/folder1')
#/kaggle/input/uta-reallife-drowsiness-dataset/Fold1_part1/Fold1_part1

zsh:1: no matches found: /kaggle/working/*


## Video Processing for Frame Extraction
Code below processes video files from the UTA directory mentioned above to extract frames at specified intervals, and save them as images (for training, validation, and testing).

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/uta-reallife-drowsiness-dataset/Fold1_part1'):
    for filename in filenames:
        participant = 1
        pathway = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))
        cam = cv2.VideoCapture(pathway)

        if not os.path.exists('/kaggle/working/frames'):
            os.mkdir('/kaggle/working/frames')

        while True:
            ret, frame = cam.read() # reading from frame
            vid = 0
            if ret:
                print("next")
                if cam.get(0) % 3000 == 0: # capturing frame every 3000 ms
                    print("captured")
                    currentframe = cam.get(5)
                    # if video is still left, continue creating images 
                    name = 'Participant_' + str(participant) + "Vid_" + str(vid) + '_Frame_' + str(currentframe) + '.jpg' # vid = 0, 5, 10; alert, neutral, drowsy
                    cv2.imwrite(name, frame) #writing the extracted images 
            else:
                break
            vid += 5

        cam.release()
        cv2.destroyAllWindows()

        participant += 1
from glob import glob

In [ ]:
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")

def save_frame(video_path, save_dir, gap=10):
    name = video_path.split("/")[-1].split(".")[0]
    save_path = os.path.join(save_dir, name)
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    idx = 0

    while True:
        ret, frame = cap.read()
        if ret == False:
            cap.release()
            break
        if idx == 0:
            cv2.imwrite(f"{save_path}/{idx}.png", frame)
        else:
            if idx % gap == 0:
                cv2.imwrite(f"{save_path}/{idx}.png", frame)
        idx += 1

if __name__ == "__main__":
    video_paths = glob("videos/*")
    save_dir = "save"

    for path in video_paths:
        save_frame(path, save_dir, gap=10)

## Frame - Class (Label) Association
Frames captured are associated with "not drowsy", "neutral", and "drowsy" classes, based on the 'vid' label within the parsed filename. They're later saved to a pandas dataframe for training, validating, and testing. 

In [ ]:
def parse_filename(filename):
    parts = filename.split('_')
    for i, part in enumerate(parts):
        if part.lower() == 'vid':
            label = int(parts[i + 1])
            if label == 0:
                return 'not_drowsy'
            elif label == 5:
                return 'neutral'
            elif label == 10:
                return 'drowsy'
            else:
                return None
    return None

In [ ]:
# ! Universalize the working directory
working_directory = '/Users/zdzilowska/Desktop/ANN final/frames'

def create_dataframe(image_dir):
    data = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith('.jpg'):
                label = parse_filename(file)
                if label:
                    data.append((os.path.join(root, file), label))
    return pd.DataFrame(data, columns=['filepath', 'label'])

df = create_dataframe(working_directory)
print(df.head(50))

                                             filepath       label
0   /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
1   /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
2   /Users/zdzilowska/Desktop/ANN final/frames/Par...     neutral
3   /Users/zdzilowska/Desktop/ANN final/frames/Par...     neutral
4   /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
5   /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
6   /Users/zdzilowska/Desktop/ANN final/frames/Par...     neutral
7   /Users/zdzilowska/Desktop/ANN final/frames/Par...     neutral
8   /Users/zdzilowska/Desktop/ANN final/frames/Par...  not_drowsy
9   /Users/zdzilowska/Desktop/ANN final/frames/Par...     neutral
10  /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
11  /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
12  /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
13  /Users/zdzilowska/Desktop/ANN final/frames/Par...      drowsy
14  /Users

## Data Preparation and Augmentation
The dataset is split into training, validation, and testing sets. The frames are then rescaled, as well as augmented for the training dataset to increase the variety of data. 

In [ ]:
# Initialization of the train, validation, and test datasets extracted from the UTA RealLife Drowsiness Dataset. 
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['label'], random_state=42)

train_datagen = ImageDataGenerator(rescale=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=0.2)
test_datagen = ImageDataGenerator(rescale=0.2)

# Artificially increases size of the training dataset; ensures a wider range of imgs. 
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 543 validated image filenames belonging to 3 classes.
Found 181 validated image filenames belonging to 3 classes.
Found 182 validated image filenames belonging to 3 classes.


## Model Definition, Compilation, and Training
The model architecture is defined using a pre-trained (on ImageNet) VGG16 base model. The top layers are excluded and the input shape is specified to match the dimensions of our input data. Custom layers are then added for the 3-class classification. To prevent the weights of the pre-trained VGG16 base model from being updated during training, we freeze all the layers of the base model, after which the model is compiled, and trained using the training and validation datasets. 

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes: 0 - not_drowsy, 5 - drowsy, 10 - neutral

model = Model(inputs=base_model.input, outputs=predictions)

# The base is freezed
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# ! Actual training
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

/Users/zdzilowska/miniconda3/envs/finalann/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.6970 - loss: 5.7686 - val_accuracy: 0.9625 - val_loss: 0.3552
Epoch 2/10
 1/16 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9062 - loss: 1.0289

/Users/zdzilowska/miniconda3/envs/finalann/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 896ms/step - accuracy: 0.9062 - loss: 1.0289 - val_accuracy: 0.9688 - val_loss: 0.2779
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.9570 - loss: 0.2605 - val_accuracy: 0.9937 - val_loss: 0.0132
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 946ms/step - accuracy: 1.0000 - loss: 4.8315e-06 - val_accuracy: 0.9875 - val_loss: 0.0259
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.9858 - loss: 0.0390 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 882ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 0.0032
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9964 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 0.0073
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 906ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 1.0000 - loss: 7.5266e-04 - val_accuracy: 1.0000 - 

In [ ]:
model.save_weights('drowsiness_detection_weights.weights.h5')

## Performance evaluation
Obtained results: 0.0897449404001236 test loss, 0.9937499761581421 test accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9966 - loss: 0.0506
Test loss: 0.0897449404001236
Test accuracy: 0.9937499761581421
